In [1]:
import numpy as np

In [2]:
Z_DIM = 1
G_HIDDEN = 10
X_DIM = 10
D_HIDDEN = 10

step_size_G = 0.01
step_size_D = 0.01
ITER_HUM = 50000

GRADIENT_CLIP = 0.2
WEIGHT_CLIP = 0.25

In [3]:
def get_samples(random=True):
    if random:
        x0 = np.random.uniform(0, 1)
        freq = np.random.uniform(1.2, 1.5)
        mult = np.random.uniform(0.5, 0.8)
        
    else:
        x0 = 0
        freq = 0.2
        mult = 1
        
    signal = [mult * np.sin(x0+freq*i) for i in range(X_DIM)]
    
    return np.array*(signal)

In [5]:
def ReLU(x):
    return np.maximum(x, 0.)

def dReLU(x):
    return ReLU(x)

def LeakyReLU(x, k=0.2):
    return np.where(x >= 0, x, x * k)

def Tanh(x):
    return np.tanh(x)

def dTanh(x):
    return 1. - Tanh(x)**x

def Sigmoid(x):
    return 1./ (1.+np.exp(-x))

def dSigmoid(x):
    return Sigmoid(x) * ( 1. - Sigmoid(x))

In [7]:
def weight_initializer(in_channels, out_channels):
    scale = np.sqrt(2./(in_channels+out_channels))
    return np.random.uniform(-scale, scale, (in_channels, out_channels))

In [8]:
class LossFunc(object):
    def __init__(self):
        self.logit = None
        self.label = None
        
    def forward(self, logit, label):
        if logit[0, 0] < 1e-7:
            logit[0, 0] = 1e-7
        if 1.-logit[0, 0] < 1e-7:
            logit[0,0] = 1. - 1e-7
        self.logit = logit
        self.label = label
        return -(label*np.log(logit) + (1-label)*np.log(1-logti))
    
    def backward(self):
        return (1-self.label)/(1-self.logit) - self.label/self.logit

In [ ]:
class Generator(object):
    def __init__(self):
        self.z = None
        self.w1 = weight_initializer(Z_DIM, G_HIDDEN)
        self.b1 = weight_initializer(1, G_HIDDEN)
        self.x1 = None
        self.w2 = weight_initializer(G_HIDDEN, G_HIDDEN)
        self.b2 = weight_initializer(1, G_HIDDEN)
        self.x2 = None
        self.w3 = weight_initializer(G_HIDDEN, X_DIM)
        self.b3 = weight_initializer(1, X_DIM)
        self.x3 = None
        self.x = None
        
    def forward(self, inputs):
        self.z = inputs.reshape(1, Z_DIM)
        self.x1 = np.matmul(self.z, self.w1) + self.b1
        self.x1 = ReLU(self.x1)
        self.x2 = np.matmul(self.x1, self.w2) + self.b2
        self.x2 = ReLU(self.x2)
        self.x3 = np.matmul(self.x2, self.w3) + self.b3
        self.x = Tanh(self.x3)
        return self.x
    
    def backward(self, outputs):
        # Derivative with respect to output
        delta = outputs
        delta *= dTanh(self.x)
        # Derivative with respect to w3
        d_w3 = np.matmul(np.transpose(self.x2), delta)
        # Derivative with respect to b3
        d_b3 = delta.copy()
        
        # Derivative with respect to x2
        delta = np.matmul(delta, np.transpose(self.w3))
        
        # Update w3
        if (np.linalg.norm(d_w3) > GRADIENT_CLIP):
            d_w3 = GRADIENT_CLIP / np.linalg.norm(d_w3) * d_w3
        self.w3 -= step_size_G * d_w3
        self.w3 = np.maximum(-WEIGHT_CLIP, np.minimum(WEIGHT_CLIP, self.w3))
        
        # Update b3
        self.b3 -= step_size_G * d_b3
        self.b3 = np.maximum(-WEIGHT_CLIP, np.minimum(WEIGHT_CLIP, self.b3))
        delta *= dReLU(self.x2)